In [1]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from scipy.special import logsumexp
import numpy as np

# Fetch dataset
zoo = fetch_ucirepo(id=111)
X = zoo.data.features
y = zoo.data.targets 
zoo_df = pd.merge(X, y, left_index=True, right_index=True)

# Inspect the dataframe to check the column names
print(zoo_df.columns)

Index(['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator',
       'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail',
       'domestic', 'catsize', 'type'],
      dtype='object')


In [8]:


# Assuming the target column is not 'class', find the correct column name
true_labels = zoo_df['type']  # replace 'animal_name' with the actual column name of your target
import warnings
warnings.filterwarnings('ignore')
# Drop any rows with missing values
zoo_df = zoo_df.dropna()

# Define the Bernoulli Mixture with Stochastic EM Algorithm class
class BernoulliMixturewSEM:
    
    def __init__(self, n_components, max_iter, batch_size=10, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.tol = tol
    
    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            for batch in self.iterate_batches():
                self.gamma = self.get_responsibilities(self.get_log_bernoullis(batch))
                self.remember_params()
                self.get_Neff()
                self.get_mu(batch)
                self.get_pi()
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.abs(self.logL - self.old_logL) < self.tol:
                break
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def iterate_batches(self):
        n_samples = len(self.x)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for start_idx in range(0, n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]
            yield self.x.iloc[batch_indices]

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
        self.old_mu = None
        self.old_pi = None
        self.old_gamma = None
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        epsilon = 1e-15
        mu_place = np.clip(mu, epsilon, 1 - epsilon)
        return np.tensordot(x, np.log(mu_place), (1,1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self, batch):
        self.mu = np.einsum('ik,id -> kd', self.gamma, batch) / self.Neff[:, None]
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

# Perform clustering and comparison
# Ensure you calculate FMI, ARI, and NMI scores for comparison

# Sample code for calculating performance metrics
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics.cluster import fowlkes_mallows_score

# Assuming 'true_labels' contains the ground truth labels
true_labels = zoo_df['type']  
# Clustering with Bernoulli Mixture with Stochastic EM Algorithm
bmm = BernoulliMixturewSEM(n_components=7, max_iter=100)
bmm.fit(X)
predicted_labels_bmm = bmm.predict(X)

# Clustering with K-Modes
from kmodes.kmodes import KModes
km = KModes(n_clusters=7, init='Huang', n_init=5, verbose=1)
predicted_labels_kmodes = km.fit_predict(X)

# Calculate performance metrics
ari_bmm = adjusted_rand_score(true_labels, predicted_labels_bmm)
nmi_bmm = normalized_mutual_info_score(true_labels, predicted_labels_bmm)
fmi_bmm = fowlkes_mallows_score(true_labels, predicted_labels_bmm)

ari_kmodes = adjusted_rand_score(true_labels, predicted_labels_kmodes)
nmi_kmodes = normalized_mutual_info_score(true_labels, predicted_labels_kmodes)
fmi_kmodes = fowlkes_mallows_score(true_labels, predicted_labels_kmodes)

# Print the results
print(f"Bernoulli Mixture with Stochastic EM Algorithm - ARI: {ari_bmm}, NMI: {nmi_bmm}, FMI: {fmi_bmm}")
print(f"K-Modes - ARI: {ari_kmodes}, NMI: {nmi_kmodes}, FMI: {fmi_kmodes}")

46.25904501111759
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 14, cost: 174.0
Run 1, iteration: 2/100, moves: 1, cost: 174.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 20, cost: 143.0
Run 2, iteration: 2/100, moves: 1, cost: 143.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 18, cost: 160.0
Run 3, iteration: 2/100, moves: 3, cost: 160.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 13, cost: 192.0
Run 4, iteration: 2/100, moves: 0, cost: 192.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 27, cost: 139.0
Run 5, iteration: 2/100, moves: 2, cost: 139.0
Best run was number 5
Bernoulli Mixture with Stochastic EM Algorithm - ARI: 0.0, NMI: 0.0, FMI: 0.482772520

<p>The Folkes-Mallows Index (FMI) is often higher than the Adjusted Rand Index (ARI) and Normalized Mutual Information Score (NMI) because it focuses solely on the similarity between clusters. FMI measures the geometric mean of precision and recall for pairs of points that are in the same or different clusters in the true and predicted clusterings.</p>

<p>In contrast, ARI and NMI consider both similarity and dissimilarity. ARI is adjusted for chance and takes into account the agreement between cluster pairs relative to random labeling. NMI measures the amount of information shared between the true and predicted clusterings, normalizing the mutual information to account for the entropy of the clusterings.</p>

<p>Therefore, FMI tends to be higher because it is less sensitive to the noise and intra-cluster variations, whereas ARI and NMI might be lower due to their adjustments and consideration of both cluster similarity and dissimilarity.</p>